# Exercise 4 -Computer Vision 
## Task 4 - Face Detection and Tracking


- [x] First import the required python libraries

In [7]:
import cv2
import time
import numpy as np
import time
import imutils


- [x] Read the OpenCV [documentation](https://docs.opencv.org/master/db/d28/tutorial_cascade_classifier.html) on Cascade classifiers which implements Viola-Jones detection algorithm. Instatntiate the cascade classifier for finding faces.
- [x] Assign variables for fps control, points collections and image collection

In [8]:
face_cascade = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')
frame_rate = 30
prev = 0
gray_prev = None
p0 = []

Complete the function which performs following
- [x] Takes a frame from video feed as the input
- [ ] Resize the frame while protecting the aspect ratio (width = 600) 
- [ ] Flip the image
- [ ] Convert the frame to grayscale image
- [x] Return grayscale image and resized image 

In [9]:
def prep(img):
    # Your code begins here
    img = imutils.resize(img, width = 600)
    img = cv2.flip(img, 1)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    # Your code ends here
    return gray, img

Complete the function which perfoms following
- [x] Takes grayscale image and resized image as the input
- [x] Detect faces in graycale image using cascade classifier. [detectMultiscale()](https://docs.opencv.org/3.4/d1/de5/classcv_1_1CascadeClassifier.html) function returns detected faces as rectangles ( Top left x coordinate, Top left y coordinate, width, height)
- [ ] Draw a rectangle around detected faces using OpenCV drawing [functions](https://docs.opencv.org/4.5.2/dc/da5/tutorial_py_drawing_functions.html)
- [ ] Slice a region of interest (ROI) from grayecale image corresponding to the detections
- [x] Extract good features to track, from OpenCV [goodFeaturesToTrack()](https://docs.opencv.org/4.5.2/d4/d8c/tutorial_py_shi_tomasi.html) function.
- [ ] The points are located with respect to the ROI. Convert them to image coordinates

In [14]:
def get_trackable_points(gray,img, p0):
    faces = face_cascade.detectMultiScale(gray, 1.1, 5)
    faces = list(faces)

    if len(faces) != 0:
        face_to_del = []
        for i in range(0, len(faces)):
            if faces[i][2] < 100 or faces[i][3] < 100:
                face_to_del.append(i)
        if len(face_to_del) > 0:
            for face in face_to_del:
                faces.pop(face)
        if len(faces) != 0:
            for (x,y,w,h) in faces:
                cv2.rectangle(img,(x,y),(x+w,y+h),(255,0,0),2)
                roi_gray = gray[y:y+h,x:x+w]

            p0 = cv2.goodFeaturesToTrack(roi_gray,maxCorners=70,qualityLevel=0.001,minDistance=5)
        
            # Your code begins here
            for p in p0:
                p[0,0] += x
                p[0,1] += y

        # Your code ends here
   
    return p0, faces, img

Complete the function which perfoms following

- [x] Use [cv2.calcOpticalFlowPyrLK()](https://docs.opencv.org/4.5.3/d4/dee/tutorial_optical_flow.html) to calculate the optical flow for tracking
- [ ] Select the valid points from p1. Note that  isFound == 1 for valid points 
- [ ] Return the valid points

In [11]:
def do_track_face(gray_prev, gray, p0):
    p1, isFound, err = cv2.calcOpticalFlowPyrLK(gray_prev, gray, p0, 
                                                            None,
                                                            winSize=(31,31),
                                                            maxLevel=10,
                                                            criteria=(cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 30, 0.03),
                                                            flags=cv2.OPTFLOW_LK_GET_MIN_EIGENVALS,
                                                            minEigThreshold=0.00025)
    
    #your code begins here
    p1 = np.delete(p1, np.where(isFound == 0), 0)

    #your code ends here
    return p1

Run the progromm
- [ ] Enter the path to video file

In [12]:
cam = cv2.VideoCapture("./Face.mp4")
if not cam.isOpened():
    raise Exception("Could not open camera/file")

p0 = []
p1 = []
while cam.isOpened:
    time_elapsed = time.time() - prev
    
    if time_elapsed > 1./frame_rate:
        
        ret_val,img = cam.read()
        
        if not ret_val:
            cam.set(cv2.CAP_PROP_POS_FRAMES, 0)  # restart video
            gray_prev = None  # previous frame
            p0 = []  # previous points
            continue
        
        prev = time.time()
        
        gray, img = prep(img)

        if len(p0) <= 10:
            p0, faces, img = get_trackable_points(gray,img, p0)
            gray_prev = gray.copy()
        
        else:
            for (x,y,w,h) in faces:
                cv2.rectangle(img,(x,y),(x+w,y+h),(255,0,0),2)
                p1 = do_track_face(gray_prev, gray, p0)
            for i in p1:
                cv2.drawMarker(img, (i[0,0], i[0,1]),[255,0,0],0)
            p0 = p1
                   
        cv2.imshow('Video feed', img)
        key = cv2.waitKey(1) & 0xFF
    if key == ord("q"):
        break
              
cv2.destroyAllWindows()

## Improved version

In [15]:
cam = cv2.VideoCapture("./Face.mp4")
if not cam.isOpened():
    raise Exception("Could not open camera/file")

p0 = []
p1 = []
frame_cnt = 0
while cam.isOpened:
    time_elapsed = time.time() - prev
    
    if time_elapsed > 1./frame_rate:
        
        ret_val,img = cam.read()
        
        if not ret_val:
            frame_cnt = 0
            cam.set(cv2.CAP_PROP_POS_FRAMES, 0)  # restart video
            p0 = []  # previous points
            continue
        
        prev = time.time()
        
        gray, img = prep(img)

        # if len(p0) <= 10:
        if frame_cnt%20 == 0:
            p0, faces, img = get_trackable_points(gray,img, p0)
        
        else:
            for (x,y,w,h) in faces: 
                cv2.rectangle(img,(x,y),(x+w,y+h),(255,0,0),2)
                p1 = do_track_face(gray_prev, gray, p0)
            for i in p1:
                cv2.drawMarker(img, (i[0,0], i[0,1]),[255,0,0],0)
            p0 = p1
        gray_prev = gray.copy()
        frame_cnt += 1
                   
        cv2.imshow('Video feed', img)
        key = cv2.waitKey(1) & 0xFF
    if key == ord("q"):
        break
              
cv2.destroyAllWindows()

# Test on webcam

In [8]:
cam = cv2.VideoCapture(0)
if not cam.isOpened():
    raise Exception("Could not open camera/file")
p0 = []
p1 = []
frame_cnt = 0
while cam.isOpened:
    time_elapsed = time.time() - prev
    
    if time_elapsed > 1./frame_rate:
        
        ret_val,img = cam.read()
        
        prev = time.time()
        
        gray, img = prep(img)

        if frame_cnt%20 == 0:
            p0, faces, img = get_trackable_points(gray,img, p0)
        
        else:
            for (x,y,w,h) in faces: 
                cv2.rectangle(img,(x,y),(x+w,y+h),(255,0,0),2)
                p1 = do_track_face(gray_prev, gray, p0)
            if(len(faces) > 0):
                for i in p1:
                    cv2.drawMarker(img, (i[0,0], i[0,1]),[255,0,0],0)
            p0 = p1
        gray_prev = gray.copy()
        frame_cnt += 1
                   
        cv2.imshow('Video feed', img)
        key = cv2.waitKey(1) & 0xFF
    if key == ord("q"):
        break
              
cam.release()
cv2.destroyAllWindows()